In [ ]:
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import fsspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt

#scientific colormaps
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
cm_data = np.loadtxt("./ScientificColormaps6/vik/vik.txt")
vik_map = LinearSegmentedColormap.from_list("vik", cm_data)
cm_data = np.loadtxt("./ScientificColormaps6/roma/roma.txt")
roma_map = LinearSegmentedColormap.from_list("roma", cm_data)
roma_map2 = LinearSegmentedColormap.from_list("roma", cm_data[-1::-1])

warnings.simplefilter('ignore') # filter some warning messages
xr.set_options(display_style="html")  #display dataset nicely 

In [ ]:
sst_anomaly_monthlyask_kubernetes import KubeCluster
from dask.distributed import Client
cluster = KubeCluster()
cluster.adapt(minimum=1, maximum=200, interval='2s', wait_count=3)
client = Client(cluster)
cluster

In [ ]:
%%time
file_location = 's3://mur-sst/zarr'

ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)

ds_sst

# subset data to region of interest

In [ ]:
subset = ds_sst.sel(lat=slice(32,48),lon=slice(-132.27,-117))

# calculate climatology using 2003-2019

In [ ]:
sst_monthly = subset.resample(time='1MS').mean('time',keep_attrs=True,skipna=False)

climatology_mean_monthly = sst_monthly.sel(time=slice('2003-01-01','2019-01-01')).groupby('time.month').mean('time',keep_attrs=True,skipna=False)

sst_anomaly_monthly = sst_monthly.groupby('time.month')-climatology_mean_monthly  #take out annual mean to remove trends

sst_anomaly_monthly = sst_anomaly_monthly.persist()

sst_anomaly_monthly

In [ ]:
mask = sst_monthly.mask.isel(time=0).persist()

In [ ]:
#axis([-132.27 -117 32.88 48.25]);
for itime in range(-100,-30):
    test = sst_anomaly_monthly.isel(time=itime)#.load()
    tem = test.where(mask<2)
    tstr = str(test.time.data)[0:10]
    plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    plt.pcolormesh(tem.lon,tem.lat,tem.analysed_sst,transform=ccrs.PlateCarree(),cmap=vik_map,vmin=-2,vmax=2)
    ax.coastlines(resolution='50m', color='black', linewidth=1)
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.STATES.with_scale('10m'))
    ax.set_extent([-132.27,-117,32,48])
    plt.colorbar(ax=ax,label='SST Anomaly (K)')
    tt=plt.text(-122,47,tstr,fontsize=16)
    plt.savefig('./figs/anom'+tstr+'.png')